In [65]:
import shutil
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [88]:
from IPython.core.display import clear_output

### DOWNLOAD PHOTO

In [2]:
url = 'https://c1.staticflickr.com/9/8094/8534020390_ee8b1b9e67_h.jpg'
file_name = 'data/images/pro/test.jpg'
r = requests.get(url, stream=True)
with open(file_name, 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
del r

### SEARACH PHOTOS

In [51]:
landscape_search_url = 'https://www.flickr.com/search/'
landscape_search_kwds = {'dimension_search_mode': 'min&height=1024&width=1024',
                         'media': 'photos',
                         'advanced': '1',
                         'text': 'landscape pro'}
headers = {'user-agent': 'Mozilla/5.0'}
r = requests.get(landscape_search_url, params=landscape_search_kwds, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [52]:
soup.select('div.content')

[]

In [53]:
soup.find_all('div', class_='photo-list-photo-interaction')

[]

### SEARCH PEOPLE

In [57]:
people_search_url = 'https://www.flickr.com/photos/12037949754@N01/'
# people_search_kwds = {'username': 'pro'}
headers = {'user-agent': 'Mozilla/5.0'}
r = requests.get(landscape_search_url)
soup = BeautifulSoup(r.text, 'html.parser')

### FLICKR API PHOTO SEARCH

In [10]:
keys_list = []
keys = {}
with open('flickr.keys') as f_open:
    for line in f_open:
        keys_list.append(line.strip('\n'))

In [14]:
for key in keys_list:
    pair = key.split(":")
    keys[pair[0]] = pair[1]

In [15]:
keys

{'Key': '933b26674bf1f7a00109f2f8eda30b52', 'Secret': '1bd86a22b5ea95bf'}

In [127]:
def search_flickr(search_tag_term, page_num=1):
    search_url = 'https://api.flickr.com/services/rest/'
    search_kwds = {'method':'flickr.photos.search',
                   'api_key':keys['Key'],
                   'tags':search_tag_term,
                   'sort':'relavance',
                   'content_type':'1',
                   'per_page':'500',
                   'page':page_num,
                   'extras': 'license, date_upload, date_taken, owner_name, original_format, last_update, geo, tags, machine_tags, o_dims, views, media, path_alias, url_o'
                  }
    r = requests.get(search_url, params=search_kwds)
    soup = BeautifulSoup(r.text, 'lxml')
    return soup, r.status_code

In [128]:
search_tag_term = 'landscape'
soup, status= search_flickr(search_tag_term)
total_pages = int(soup.photos.get('pages'))
print 'Status: {}'.format(status)

Status: 200


In [ ]:
picture_soup_data = []
for i in range(1, total_pages+1):
    soup, status = search_flickr(search_tag_term, page_num=i)
    if int(status) != 200:
        print "Error on Page {}: Status Code {}".format(i, status)
        print "Print soup to determine problem."
        break
    if i == 1:
        print "Begining Search for '{}': {} Pages Found".format(search_tag_term, total_pages)
    picture_soup_data.extend(soup.findAll('photo'))
    if i%50==0:
        print "{} Pages Complete".format(i)
    if i == total_pages:
        print "Search Complete"
        print "-"*30

Begining Search for 'landscape': 980 Pages Found


In [ ]:
len(picture_soup_data)

In [107]:
columns = ['id', 'title', 'owner', 'secret', 'farm', 'server', 'ispublic']
int_columns = ['id', 'farm', 'server', 'ispublic']

In [112]:
picture_data = []
for i, picture in enumerate(pictures):
    picture_data.append([int(picture.get(name)) if name in int_columns else picture.get(name) for name in columns])
    if i%50000==0:
        print float(i)/len(pictures)*100," percent complete"

0.0  percent complete
10.0813565473  percent complete
20.1627130947  percent complete
30.244069642  percent complete
40.3254261893  percent complete
50.4067827367  percent complete
60.488139284  percent complete
70.5694958314  percent complete
80.6508523787  percent complete
90.732208926  percent complete


In [114]:
df = pd.DataFrame(data=picture_data, columns=columns)

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495965 entries, 0 to 495964
Data columns (total 7 columns):
id          495965 non-null int64
title       495965 non-null object
owner       495965 non-null object
secret      495965 non-null object
farm        495965 non-null int64
server      495965 non-null int64
ispublic    495965 non-null int64
dtypes: int64(4), object(3)
memory usage: 26.5+ MB


In [118]:
df.head()

,id,title,owner,secret,farm,server,ispublic
0,27797327313,Mr. Tanaka｜田中先生,67186028@N06,677ebd93b4,9,8700,1
1,27774358924,The Church on the Hill,16946718@N05,eed93a7b61,9,8769,1
2,27796175404,N160718_022.jpg,61301115@N03,ba41305f68,9,8273,1
3,28130293950,DP2M0766_edited-1.jpg,142536946@N03,12e891a9bf,8,7622,1
4,27797281933,kale,145189424@N02,d1189a576b,8,7470,1


In [5]:
url = 'https://farm{farm_id}.staticflickr.com/{server_id}/{ID}_{secret}_{mstzb}.jpg'
url_full = url.format(farm_id=farm_id, server_id=server_id, ID=ID, secret=secret, mstzb=mstzb)

NameError: name 'farm_id' is not defined